In [ ]:
import numpy as np
from IPython.display import HTML
import random

import pandas as pd
from pyproj import Geod


In [ ]:
# latitude and longtitude of each place
# pos_list = [
#     [43.06417, 141.34694, "Sapporo"], 
#     [38.26889, 140.87194, "Sendai"], 
#     [36.59444, 136.62556, "Kanazawa"], 
#     [35.68944, 139.69167, "Tokyo"], 
#     [35.18028, 136.90667, "Nagoya"]
# ]
departure = [[40.6643, -73.9385, "New York"]] #departure
pos_list = [
    [38.9041, -77.0171, "Washington"], 
    [42.332, -71.0202, "Boston"],
    [45.4208, -75.6945, "Otawwa"],
    [43.7166, -79.3407, "Tront"], 
    [46.8127, -71.2199, "Quebec"]
]
point_num = len(pos_list)

# agents
agents = ["Agent1", "Agent2"]
agent_num = len(agents)

In [ ]:
import folium

# function to show results on a map
def visualize(df):
    # map size
    f = folium.Figure(width=600, height=600)

    # create the map with center latitude and longtitude
    center_lat = 40.000
    center_log = -75.000
    map = folium.Map(location=[center_lat,center_log], zoom_start=5)

    # put pins on the map
    for pos in pos_list:
        folium.Marker([pos[0], pos[1]], popup=pos[2]).add_to(map)
        
    # the order of the places to visit
    order_list = []
    for i in range(point_num):
        for j in range(point_num):
            if df[i][j] == 1:
                order_list.append(j)

    # draw a line between two places depending on the order
    for i in range(-1, len(order_list)-1):
        folium.PolyLine(locations=[pos_list[order_list[i]][:2], pos_list[order_list[i+1]][:2]],
                        weight=3,color='#4169e1').add_to(map)
    f.add_child(map)
    return f

In [ ]:
# function to calculate distance from latitude and longtitude
def calc_distance(point_x1, point_y1, point_x2, point_y2):
    g = Geod(ellps='WGS84')
    # get angles and a distance between two places
    angle_1, angle_2, distance = g.inv(point_x1, point_y1, point_x2, point_y2)
    distance /= 1000
    return distance

In [ ]:
from pyqubo import solve_qubo, Array, Placeholder

# create spins
x = Array.create('x', shape=(agent_num, point_num), vartype='BINARY')

In [ ]:
# caluculate distance from departure
pos_len = []
distance_dict = {}
for i in range(point_num):
    distance = calc_distance(pos_list[0][1], pos_list[0][0], pos_list[i][1], pos_list[i][0])
    distance_dict[i] = distance

print(distance_dict)

In [ ]:
import itertools
# combination of two places
combinations = [(i,j) for i,j in itertools.product(range(point_num), repeat=2) if i != j]

H1 = 0

# roup of each agent
for a in range(agent_num):
    for p in range(point_num):
        H1 += distance_dict[p] * x[a, p]
    

In [ ]:
# Only one place is visited at a time
H2 = 0
for j in range(point_num):
    H2_1 = 0
    for i in range(point_num):
        H2_1 += x[a,p]
    H2 += (H2_1 - 1) **2

In [ ]:
# Each agent visits one place only once
H3 = 0
for a in range(agent_num):
    H3_1 = 0
    for j in range(point_num):
        H3_1 += x[a, j]
    H3 += (H3_1 -1)**2

In [ ]:
H = Placeholder('param_1') * H1 + Placeholder('param_2') * H2 + Placeholder('param_3') * H3

model = H.compile()
feed_dict = {'param_1': 0.001, 'param_2': 1.5, 'param_3': 1.5}
qubo, offset = model.to_qubo(feed_dict=feed_dict)

In [ ]:
# D-Wave
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

# setting for D-Wave
bqm = dimod.BQM(qubo, 'BINARY')

# token = xxxxx
# endpoint = xxxxx

# sampling
dw_sampler = DWaveSampler(solver='Advantage2_prototype2.5', token=token, endpoint=endpoint)
sampler = EmbeddingComposite(dw_sampler)
response = sampler.sample(bqm, num_reads=100)


In [ ]:
# # show results
# for sample, energy in response.data(['sample', 'energy']):
#     print(sample, 'Energy', energy)

In [ ]:
decoded_sampleset = model.decode_sampleset(response, feed_dict=feed_dict)
best_sample = min(decoded_sampleset, key=lambda s: s.energy)
# print(best_sample)

In [ ]:
# create a list to show the result
list1 = []
for j in range(point_num):
    list2 = []
    for i in range(point_num):
        list2.append(best_sample.sample[f'x[{i}][{j}]'])
    list1.append(list2)

In [ ]:
# print the result
df = pd.DataFrame(list1)
index_list = []
for l in pos_list:
    index_list.append(l[2])
df.index = index_list
print(df)

In [ ]:
#  visualize the result
visualize(df)